 "**Meeting Assistant AI**" app to allow live recording of the meeting along with maintaining the existing features (handling PDF, audio, and chatbot), you can use Python's sounddevice and wavio libraries to capture live audio, save it, and then use the captured audio for transcription and summarization.

**Step 1:** Install Additional Dependencies

We will need to install the sounddevice and wavio libraries for live audio recording.

In [6]:
!pip install sounddevice
!pip install wavio
!pip install requests
!pip install PyPDF2
!pip install pydub
!pip install SpeechRecognition
!apt-get install ffmpeg  # Required for handling audio formats like mp3


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


**Step 2**: Import the Required Libraries

You need to import the sounddevice library for live recording, wavio to save the recorded audio, and the rest of the libraries for the existing functionalities.

In [7]:
!apt-get install libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [8]:
import requests
import json
import PyPDF2
from io import BytesIO
from google.colab import files
from pydub import AudioSegment
import speech_recognition as sr
import sounddevice as sd
import wavio
import numpy as np


**Step 3:** Live Recording Function

Here, we define a function that records live audio from the user's microphone and saves it as a .wav file. You can adjust the duration and sampling rate as per your needs.

In [11]:
# Function to record live audio
def record_live_audio(duration=60, sample_rate=44100):
    print("Recording audio for", duration, "seconds...")

    # Record audio with specified duration and sample rate
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
    sd.wait()  # Wait until the recording is finished

    # Save the recording as a WAV file
    wavio.write("live_recording.wav", recording, sample_rate, sampwidth=2)
    print("Recording saved as 'live_recording.wav'")
    return "live_recording.wav"


This function records the user's voice for a specified duration (default 60 seconds) and saves the audio as live_recording.wav. You can change the duration by passing a different value when calling the function.

**Step 4:** Modify Transcript Extraction from Audio

This function will now handle both uploaded audio files and live recorded audio for transcription.

In [12]:
# Function to convert audio to text using Google Speech-to-Text API
def transcribe_audio(audio_file):
    recognizer = sr.Recognizer()

    # Load the audio file into the recognizer
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)  # Capture the entire audio file

    try:
        # Use Google Web Speech API to transcribe the audio file
        transcript_text = recognizer.recognize_google(audio)
        print("Transcription Success! Transcript:\n", transcript_text)
        return transcript_text
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand the audio")

    return None


**Step 5:** Ask User for Input Type (PDF, Audio, or Live Recording)

Now, we modify the input selection to include live recording as an option, in addition to PDF and audio file uploads.

In [13]:
# Ask the user if they want to upload a PDF, an audio file, or do a live recording
file_type = input("Would you like to upload a 'PDF', 'Audio', or 'Live' recording? ").strip().lower()

# Initialize an empty transcript variable
transcript = None

if file_type == 'pdf':
    # Handle PDF input
    uploaded = files.upload()
    for file_name in uploaded.keys():
        with open(file_name, 'rb') as f:
            transcript = f.read()

    transcript_stream = BytesIO(transcript)
    pdf_reader = PyPDF2.PdfReader(transcript_stream)

    text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()

    print("Meeting Transcript Captured from PDF:\n", text)

elif file_type == 'audio':
    # Handle audio input
    audio_uploaded = files.upload()
    for audio_file_name in audio_uploaded.keys():
        audio = AudioSegment.from_file(audio_file_name)
        audio.export("transcript.wav", format="wav")
        transcript = transcribe_audio("transcript.wav")

elif file_type == 'live':
    # Handle live recording input
    audio_file = record_live_audio(duration=60)  # Adjust duration as needed
    transcript = transcribe_audio(audio_file)

else:
    print("Invalid choice. Please upload either 'PDF', 'Audio', or do a 'Live' recording.")


Would you like to upload a 'PDF', 'Audio', or 'Live' recording? live
Recording audio for 60 seconds...


PortAudioError: Error querying device -1

**Step 6:** Use the Transcript for Summarization

This step remains unchanged. Whether the transcript comes from a PDF, uploaded audio, or live recording, it will be summarized using the Google Gemini API.

In [ ]:
# Function to summarize the meeting transcript
def summarize_meeting(transcript_text):
    data = {
        "contents": [
            {
                "parts": [
                    {"text": transcript_text}
                ]
            }
        ]
    }

    try:
        # Make the POST request to the Summarization API
        response = requests.post(summarization_url, headers=headers, json=data)

        # Print the raw response for debugging
        print("API Response Status Code:", response.status_code)
        print("API Response Content:", response.text)

        # Check if the request was successful
        if response.status_code == 200:
            return response.json()  # Return the parsed JSON response
        else:
            print(f"Failed to summarize. Status Code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None


# Call the summarization function if transcript exists
if transcript:
    summary_response = summarize_meeting(transcript)
    if summary_response and "candidates" in summary_response:
        try:
            summary = summary_response["candidates"][0]["content"]["parts"][0]["text"]
            print("Meeting Summary:\n", summary)
        except (IndexError, KeyError) as e:
            print(f"Error extracting summary from the response: {e}")
else:
    print("No transcript available for summarization.")


**Step 7:** Chatbot for Asking Questions

Finally, the chatbot functionality remains the same, allowing users to ask questions based on the generated summary.

In [ ]:
# Function to ask questions based on the summary
def ask_question(question, summary_text):
    prompt_data = {
        "contents": [
            {
                "parts": [
                    {"text": f"{question}: {summary_text}"}
                ]
            }
        ]
    }

    try:
        response = requests.post(prompt_url, headers=headers, json=prompt_data)
        print("Prompt API Response Status Code:", response.status_code)
        print("Prompt API Response Content:", response.text)

        if response.status_code == 200:
            return response.json()  # Return the parsed JSON response
        else:
            print(f"Failed to get an answer. Status Code: {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Chatbot loop for asking questions
if summary:
    while True:
        user_query = input("Ask a question about the meeting summary (or type 'exit' to quit): ")

        if user_query.lower() == 'exit':
            print("Exiting chatbot...")
            break

        user_prompt_response = ask_question(user_query, summary)

        if user_prompt_response and "candidates" in user_prompt_response:
            try:
                answer = user_prompt_response["candidates"][0]["content"]["parts"][0]["text"]
                print(f"Answer to your query:\n{answer}")
            except (IndexError, KeyError) as e:
                print(f"Error accessing the answer in the response: {e}")
        else:
            print("Response format is incorrect or no answer available.")
else:
    print("No summary available to ask questions.")


How the Extended "**Meeting Assistant AI**" Works:

    Install Dependencies: Includes the new sounddevice and wavio libraries for recording live audio.
    Live Recording: Added functionality to record live audio from the user's microphone.
    Choose Input Type: Users can now select between uploading a PDF, an audio file, or recording live audio.
    Transcription & Summarization: The selected input (live audio, uploaded audio, or PDF) is transcribed and summarized.
    Chatbot for Queries: Users can interact with the app by asking questions about the summarized meeting.